1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래 걸림
3. 임베딩 -> 벡터 데이터 베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달


https://python.langchain.com/docs/integrations/document_loaders/microsoft_word/#using-azure-ai-document-intelligence

https://python.langchain.com/docs/how_to/recursive_text_splitter/#splitting-text-from-languages-without-word-boundaries

In [24]:
# pip install --upgrade --quiet  docx2txt langchain_community

In [25]:
# pip install -qU langchain-text-splitters

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap=200, # 오버랩을 함으로써 문서를 겹치게함. 문맥을 전달할 수 있음.
)

loader = Docx2txtLoader("./tax.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)

In [8]:
len(document_list)

187

https://python.langchain.com/docs/integrations/text_embedding/

In [ ]:
# 3. 임베딩 -> 벡터 데이터 베이스에 저장

from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [4]:
embedding

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x00000204107A10D0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000002040F826BD0>, model='text-embedding-3-large', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

https://python.langchain.com/docs/integrations/vectorstores/chroma/#basic-initialization

우리 쪼개논 다큐먼트 리스트를 임베딩을 이용해서 저장

In [16]:
from langchain_chroma import Chroma

# database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='chroma-tax', persist_directory="./chroma")

database = Chroma(collection_name='chroma-tax', persist_directory="./chroma", embedding_function=embedding)

similarity_search: 유사도 검색, 임베딩을 활용해서 유사검색을 해서 답변을 가져옴

In [9]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
#retrieved_docs = database.similarity_search(query, k=2)

In [ ]:
#retrieved_docs

[Document(id='eb910c64-50ef-4c0b-91a2-717d66bd6335', metadata={'source': './tax.docx'}, page_content='1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다) 또는 해당 과세기간에 대통령령으로 정하는 총수입금액의 합계액이 2천만원 이하인 자의 주택임대소득(2018년 12월 31일 이전에 끝나는 과세기간까지 발생하는 소득으로 한정한다). 이 경우 주택 수의 계산 및 주택임대소득의 산정 등 필요한 사항은 대통령령으로 정한다.\n\n다. 대통령령으로 정하는 농어가부업소득\n\n라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득\n\n마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양도로 발생하는 소득으로서 연 600만원 이하의 금액. 이 경우 조림기간 및 세액의 계산 등 필요한 사항은 대통령령으로 정한다.\n\n바. 대통령령으로 정하는 작물재배업에서 발생하는 소득\n\n사. 대통령령으로 정하는 어로어업 또는 양식어업에서 발생하는 소득\n\n3. 근로소득과 퇴직소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 대통령령으로 정하는 복무 중인 병(兵)이 받는 급여\n\n나. 법률에 따라 동원된 사람이 그 동원 직장에서 받는 급여\n\n다. 「산업재해보상보험법」에 따라 수급권자가 받는 요양급여, 휴업급여, 장해급여, 간병급여, 유족급여, 유족특별급여, 장해특별급여, 장의비 또는 근로의 제공으로 인한 부상ㆍ질병ㆍ사망과 관련하여 근로자나 그 유족이 받는 배상ㆍ보상 또는 위자(慰藉)의 성질이 있는 급여\n\n라. 「근로기준법」 또는 「선원법」에 따라 근로자ㆍ선원 및 그 유족이 받는 요양보상금, 휴업보상금

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [ ]:
'''
prompt = f"""[Indentity]
- 당신은 최고의 한국 소득세 전문가 입니다
- [Context]를 참고해서 사용자의 질문에 답변해주세요

[Context]
{retrieved_docs}

Question: {query}
"""'
'''

'\nprompt = f"""[Indentity]\n- 당신은 최고의 한국 소득세 전문가 입니다\n- [Context]를 참고해서 사용자의 질문에 답변해주세요\n\n[Context]\n{retrieved_docs}\n\nQuestion: {query}\n"""\'\n'

In [ ]:
# ai_message = llm.invoke(prompt)

In [ ]:
# ai_message.content

'연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 근로소득공제를 비롯한 다양한 공제 항목들을 먼저 고려해야 합니다. 여기서는 대략적인 계산 방법을 설명드리겠습니다.\n\n1. **총급여액**: 먼저, 연봉 5천만원은 총급여액입니다.\n\n2. **근로소득공제**: 근로소득공제는 아래와 같은 공식으로 계산됩니다.\n   - 최대 1,500만원 이하: 70% (공제한도: 1,050만원)\n   - 1,500만원 초과 ~ 4,500만원 이하: 1,050만원 + (금액-1,500만원)의 40%\n   - 4,500만원 초과 ~ 1억원 이하: 2,550만원 + (금액-4,500만원)의 15%\n   - 1억원 초과: 3,750만원 + (금액-1억원)의 5%\n\n   5천만원의 경우: \n   \\[\n   2,550만원 + (5천만원 - 4,500만원) \\times 15\\% = 2,625만원\n   \\]\n\n3. **소득세 과세표준**: \n   - 과세표준 = 총급여액 - 근로소득공제\n   - 5,000만원 - 2,625만원 = 2,375만원\n\n4. **소득세율**: 과세표준에 따라 소득세율을 적용합니다.\n   - 1,200만원 이하: 6%\n   - 1,200만원 초과 ~ 4,600만원 이하: 15% (누진공제 108만원)\n   - ... (이후 구간 생략)\n\n   2,375만원은 1,200만원 초과 ~ 4,600만원 이하 구간에 속하므로:\n   \\[\n   (2,375만원 \\times 15\\%) - 108만원 = 246.25만원 - 108만원 = 138.25만원\n   \\]\n\n5. **결론**: \n   - 이 외에 추가적으로 적용 가능한 세액공제 등 개인별 상황에 따라 최종 소득세는 달라질 수 있습니다. 대략적인 계산으로 138.25만원의 소득세가 부과될 것으로 예상됩니다.\n\n위 계산은 대략적인 절차이며, 더 정확한 소득세 계산을 위해서는 각 개인의 공제 항목, 세액공제, 추가 소득 등을 고려해야 합니다. 세부적인 계산이

In [35]:
# pip install -U langchain langchainhub --quiet

In [12]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

c:\Users\노준영\Desktop\LLM_RAG\new_env\Lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [37]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [17]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt":prompt}
)

In [19]:
ai_message = qa_chain.invoke({"query":query})

In [20]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': "I'm sorry, but the provided context does not contain information about the specific calculation of income tax for a salary of 50 million KRW."}